<a href="https://colab.research.google.com/github/nikhil-mathews/MastersPr_Predicting-Human-Pathogen-PPIs-using-Natural-Language-Processing-methods/blob/main/4.Final_model/hv_final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This is where the model is trained on a single train - test pair

In [ ]:
import pandas as pd
#Google colab does not have pickle
try:
  import pickle5 as pickle
except:
  !pip install pickle5
  import pickle5 as pickle
import os
import seaborn as sns

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, GlobalMaxPooling1D,Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding,Concatenate
from keras.models import Model,load_model
from sklearn.metrics import roc_auc_score,confusion_matrix,roc_curve, auc
from numpy import random
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.metrics import AUC

import sys
sys.path.insert(0,'/content/drive/MyDrive/ML_Data/')
import functions as f

directory = '/content/drive/MyDrive/ML_Data/'

     |████████████████████████████████| 133kB 11.4MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219256 sha256=e7f91d522db61dd6ebf4520af801df59a3ced4c9a67d50a9cbde8eebe18e01ed
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


### Pre-processed HV-PPI dataset looks like this

In [ ]:
def load(name):
    try:
        with open(directory+''+name+'.pickle', 'rb') as handle:
            return pickle.load(handle)
    except:
        with open(directory+''+name+'.pkl', 'rb') as handle:
            return pickle.load(handle)

df_train = load('HV_train')
df_train = df_train.reset_index(drop=True)
df_train

,Human,Virus,Label
0,"[M, E, R, A, V, P, L, A, V, P, L, G, Q, T, E, ...","[M, N, E, V, K, C, V, F, E, T, K, L, S, P, G, ...",0
1,"[M, E, A, A, A, Q, F, F, V, E, S, P, D, V, V, ...","[M, A, E, E, Q, A, Y, H, V, N, K, G, L, E, C, ...",0
2,"[M, D, A, N, S, K, D, K, P, P, E, T, K, E, S, ...","[M, K, L, L, V, G, I, L, V, A, V, C, L, H, Q, ...",0
3,"[M, V, T, V, G, N, Y, C, E, A, E, G, P, V, G, ...","[M, A, R, F, E, D, P, T, R, R, P, Y, K, L, P, ...",0
4,"[M, H, D, E, C, T, P, Q, Q, T, M, S, S, I, Q, ...","[M, T, A, T, P, L, T, N, L, F, L, R, A, P, D, ...",0
...,...,...,...
416890,"[M, E, Q, Q, D, Q, S, M, K, E, G, R, L, T, L, ...","[M, A, F, Y, L, P, D, W, S, C, C, G, L, W, L, ...",0
416891,"[M, M, A, A, E, A, G, S, E, E, G, G, P, V, T, ...","[M, A, K, A, T, G, R, Y, N, L, V, P, P, K, K, ...",0
416892,"[M, N, G, V, S, E, G, T, R, G, C, S, D, R, Q, ...","[M, L, E, P, L, Q, I, L, S, I, C, S, F, I, L, ...",0
416893,"[M, K, S, L, K, S, R, L, R, R, Q, D, V, P, G, ...","[M, R, P, G, R, P, L, A, G, F, Y, A, T, L, R, ...",0


### They must be converted into a format usable by the model

In [ ]:
df_train = df_train.rename(columns={'Virus':'Yersinia'})
def join(df):
  df["Joined"] = [df.loc[row]['Human']+df.loc[row]['Yersinia'] for row in range(df.shape[0])]
  return df
join(df_train)
df_train

,Human,Yersinia,Label,Joined
0,"[M, E, R, A, V, P, L, A, V, P, L, G, Q, T, E, ...","[M, N, E, V, K, C, V, F, E, T, K, L, S, P, G, ...",0,"[M, E, R, A, V, P, L, A, V, P, L, G, Q, T, E, ..."
1,"[M, E, A, A, A, Q, F, F, V, E, S, P, D, V, V, ...","[M, A, E, E, Q, A, Y, H, V, N, K, G, L, E, C, ...",0,"[M, E, A, A, A, Q, F, F, V, E, S, P, D, V, V, ..."
2,"[M, D, A, N, S, K, D, K, P, P, E, T, K, E, S, ...","[M, K, L, L, V, G, I, L, V, A, V, C, L, H, Q, ...",0,"[M, D, A, N, S, K, D, K, P, P, E, T, K, E, S, ..."
3,"[M, V, T, V, G, N, Y, C, E, A, E, G, P, V, G, ...","[M, A, R, F, E, D, P, T, R, R, P, Y, K, L, P, ...",0,"[M, V, T, V, G, N, Y, C, E, A, E, G, P, V, G, ..."
4,"[M, H, D, E, C, T, P, Q, Q, T, M, S, S, I, Q, ...","[M, T, A, T, P, L, T, N, L, F, L, R, A, P, D, ...",0,"[M, H, D, E, C, T, P, Q, Q, T, M, S, S, I, Q, ..."
...,...,...,...,...
416890,"[M, E, Q, Q, D, Q, S, M, K, E, G, R, L, T, L, ...","[M, A, F, Y, L, P, D, W, S, C, C, G, L, W, L, ...",0,"[M, E, Q, Q, D, Q, S, M, K, E, G, R, L, T, L, ..."
416891,"[M, M, A, A, E, A, G, S, E, E, G, G, P, V, T, ...","[M, A, K, A, T, G, R, Y, N, L, V, P, P, K, K, ...",0,"[M, M, A, A, E, A, G, S, E, E, G, G, P, V, T, ..."
416892,"[M, N, G, V, S, E, G, T, R, G, C, S, D, R, Q, ...","[M, L, E, P, L, Q, I, L, S, I, C, S, F, I, L, ...",0,"[M, N, G, V, S, E, G, T, R, G, C, S, D, R, Q, ..."
416893,"[M, K, S, L, K, S, R, L, R, R, Q, D, V, P, G, ...","[M, R, P, G, R, P, L, A, G, F, Y, A, T, L, R, ...",0,"[M, K, S, L, K, S, R, L, R, R, Q, D, V, P, G, ..."


### Split into 50 datasets, converted to 5D and stored.

This takes time

In [ ]:
#f.save(df_train,'HV_train_1D')
df_train = f.load('HV_train_1D')

multiple = df_train.shape[0]//50


"""
not perfect. one datapoint is repeated across each subset, and some are lost.
but considering the overall size, impact of this on result is zero
"""
for i in range(50):
  print(i+1)
  df = df_train.loc[multiple*i:multiple*(i+1)].reset_index(drop=True)
  f.combine_AC(df,5)
  #f.save(df,'HV_train_'+str(i)+'')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


### 25GB RAM can only process half the training data for tokenizer creation

In [ ]:
train_5D = f.load('HV_train_0')
for i in range(1,25):
  train_5D = pd.concat([train_5D,f.load('HV_train_'+str(i)+'')]).reset_index(drop=True)
f.create_tokenizers(train_5D)

Saved tokenizers as doubleip_tkrs
Saved tokenizer as join_tkr


### Test data converted to 5D and saved, preprocessed and saved

In [ ]:
df_test = f.load('HV_test_1D')
f.combine_AC(df_test,5)
#f.save(df_test,'HV_test_5D')

In [ ]:
df_test = f.load('HV_test_5D')
tests = f.preprocess(df_test,saveTokrs=False)
#f.save(tests,'tests')

Preprocessing...


validation data during training

In [ ]:
df_test_5D = f.load('HV_test_5D')
val_tests = df_test_5D.loc[:1500]
#f.save(val_tests,'val_tests')

### The model is created and saved

In [ ]:
EMBEDDING_DIM_5D = 50
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 32
EPOCHS = 1
DROP = 0.5
threshold = 0.918
MAX_SEQUENCE_LENGTH_5D_J = 2000
MAX_SEQUENCE_LENGTH_5D_dIP = 1000
num_words_5D_J = 1000000
num_words_5D = 500000

x1_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
x2_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)
x3_join = f.transf_model(MAX_SEQUENCE_LENGTH_5D_J,num_words_5D_J,5,0.9)

x1_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x2_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x3_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x4_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x5_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)
x6_doubleip = f.conv_model(MAX_SEQUENCE_LENGTH_5D_dIP,EMBEDDING_DIM_5D,num_words_5D,DROP)

concatenator = Concatenate(axis=1)
x = concatenator([x1_join.output, x2_join.output, x3_join.output, x1_doubleip.output, x2_doubleip.output, x3_doubleip.output, x4_doubleip.output, x5_doubleip.output, x6_doubleip.output])
x = Dense(256)(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid",name="Final")(x)
model_HV = Model(inputs=[x1_join.input, x2_join.input, x3_join.input, x1_doubleip.input, x2_doubleip.input, x3_doubleip.input, x4_doubleip.input, x5_doubleip.input, x6_doubleip.input], outputs=output)

model_HV.compile(loss='binary_crossentropy', optimizer='adam', metrics=AUC())

#model_HV.save(directory+'model_HV')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_Data/model_HV/assets


### Trained on 2 epochs

In [ ]:
df_test = f.load('val_tests')
val_tests = f.preprocess(df_test,saveTokrs=False)
model_HV = load_model(directory+'model_HV')
names = ['HV_train_'+str(i) for i in range(50)]

for name in names+names:
  print(name)
  df_train = f.load(name)
  trains = f.preprocess(df_train,saveTokrs=False)
  model_HV.fit(trains, df_train['Label'].values, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(val_tests,df_test['Label'].values))

Preprocessing...
HV_train_0
Preprocessing...
261/261 [==============================] - 212s 793ms/step - loss: 0.1726 - auc_2: 0.6064 - val_loss: 0.1784 - val_auc_2: 0.7748
HV_train_1
Preprocessing...
261/261 [==============================] - 202s 772ms/step - loss: 0.1543 - auc_2: 0.7383 - val_loss: 0.1385 - val_auc_2: 0.8294
HV_train_2
Preprocessing...
261/261 [==============================] - 200s 768ms/step - loss: 0.1388 - auc_2: 0.7802 - val_loss: 0.1342 - val_auc_2: 0.8429
HV_train_3
Preprocessing...
261/261 [==============================] - 196s 749ms/step - loss: 0.1307 - auc_2: 0.8183 - val_loss: 0.1379 - val_auc_2: 0.8519
HV_train_4
Preprocessing...
261/261 [==============================] - 210s 803ms/step - loss: 0.1327 - auc_2: 0.8269 - val_loss: 0.1327 - val_auc_2: 0.8563
HV_train_5
Preprocessing...
261/261 [==============================] - 216s 828ms/step - loss: 0.1206 - auc_2: 0.8408 - val_loss: 0.1275 - val_auc_2: 0.8803
HV_train_6
Preprocessing...
261/261 [====

In [ ]:
#model_HV.save(directory+'model_HV')
model_HV = load_model(directory+'model_HV')
df_test = f.load('HV_test_5D')
roc_auc_score(df_test['Label'].values, model_HV.predict(f.load('tests')))

0.9380718790442475